# Scikit-Learn Data Preprocessor
## Using TITANIC_VIEW from SAP Datasphere. This view has 861 records

## Install fedml_azure package

In [ ]:
pip install fedml_azure --force-reinstall

## Import the libraries needed in this notebook

In [ ]:
from fedml_azure import DwcAzureTrain

## Set up
### Creating a Training object and setting the workspace, compute target, and environment.

Before running the below cell, ensure that you have a workspace and replace the subscription_id, resource_group, and workspace_name with your information.

The whl file for the fedml_azure library must be passed to the pip_wheel_files key in the environment_args and to use scikit-learn, you must pass the name to conda_packages as well.


In [ ]:
#creation of training object and creating workspace in constructor.

training = DwcAzureTrain(
                    workspace_args={"subscription_id": '<subscription_id>',
                                        "resource_group": '<resource_group>',
                                        "workspace_name": '<workspace_name>'
                                        },
                    experiment_args={'name':'test-2'},
                    environment_type='CondaPackageEnvironment',
                    environment_args={'name':'test-env-pre','conda_packages':['scikit-learn'],'pip_packages':['fedml_azure']},
                    compute_type='AmlComputeCluster',
                    compute_args={'vm_size':'Standard_D12_v2',
                        'vm_priority':'lowpriority',
                        'compute_name':'cpu-pre-clu',
                        'min_nodes':0,
                        'max_nodes':1,
                        'idle_seconds_before_scaledown':1700
                    })


### Then, we need to generate the run config. This is needed to package the configuration specified so we can submit a job for training. 

Before running the following cell, you should have a config.json file with the specified values to allow you to access to SAP Datasphere. Provide this file path to config_file_path in the below cell.

You should also have the follow view TITANIC_VIEW created in your SAP Datasphere. To gather this data, please refer to https://www.kaggle.com/c/titanic/data and download the train.csv file.

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py

In [ ]:
#generating the run config
src=training.generate_run_config(config_file_path='dwc_configs/config.json',
                          config_args={
                                          'source_directory':'Scikit-Learn-Preprocessor',
                                          'script':'preprocessor_script.py',
                                          'arguments':['--model_file_name','regression.pkl', '--table_name', 'TITANIC_VIEW']
                                          }
                            )

### Submitting the job for training
setting is_download to true so the output of the model is downloaded. 
This output is used in the hyperparameter tuning model.

In [ ]:
#submitting the training run
run=training.submit_run(src, is_download=True)

## Register the model for deployment

In [ ]:
model=training.register_model(run=run,
                           model_args={'model_name':'sklearn_preprocessor_model',
                                       'model_path':'outputs/regression.pkl'},
                            resource_config_args={'cpu':1, 'memory_in_gb':0.5},
                            is_sklearn_model=True
                           )
print('Name:', model.name)
print('Version:', model.version)